This is for testing models with 3 specific layers

## Import libraries and set up functions
Starting directory is this "..\\asset" from this notebook code

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
tf.config.run_functions_eagerly(True)
tf.data.experimental.enable_debug_mode()
import cv2
import matplotlib.pyplot as plt
import random
from keras.backend import clear_session
from keras.backend import get_session
import time
import math
from datetime import timedelta
# import mediapipe as mp
import csv
import pandas as pd
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau

os.chdir("..\\..\\asset") # change working directory

In [2]:
emotionList = ('Neutral', 'Happy', 'Sad', 'Surprise', 'Fear', 'Disgust', 'Anger', 'Contempt')
TRAINANNOTATIONPATH = "train_set\\annotations\\"
TRAINIMAGEPATH = "train_set\\images\\"
TRAINLANDMARKPATH = "train_set\\landmarks\\"
TESTANNOTATIONPATH = "val_set\\annotations\\"
TESTIMAGEPATH = "val_set\\images\\"
TESTLANDMARKPATH = "val_set\\landmarks\\"

def LoadAllImageNames(filePath, limit = 0, catLimit = []):
  """Load the images names and label in tuple format (label, image name)

  Args:
      filePath (str): directory of image folder
      limit (int, optional): max number of image to load. Defaults to 0.
      catLimit (list<int>, optional): array of image count limit for each class. Defaults to []*8.

  Returns:
      list<str>: shuffled list of image names
  """
  limitCounter = [0,0,0,0,0,0,0,0]
  dataSet = []
  loadCounter = 0

  annotationPath = TRAINANNOTATIONPATH if ("train" in filePath) else TESTANNOTATIONPATH
  for file in os.scandir(filePath):
    if (limit > 0 and loadCounter > limit):
      break
  
    name = file.name[:-4] # file name w/o file extension
    if "_landmark" in name:
      name = name[:-9]
    data = np.load("{}{}_exp.npy".format(annotationPath, name)) # 
    label = int(data.item(0))

    if len(catLimit) > 0 and limitCounter[label] >= catLimit[label]:
        continue
    limitCounter[label] += 1
    loadCounter += 1

    dataSet.append((label, file.name))
    if (loadCounter%10000==0):
      print("Files loaded:{}".format(loadCounter))
  
  print("Total images loaded: ", loadCounter)
  print("Images Loaded: ", limitCounter)
  random.shuffle(dataSet)
  return dataSet

# Load the pixels of a picture to numpy.ndarray format. false for test set, true for training set
# Return image in RGB format
def LoadImage(imagePath, imageName, normalize = True):
  """Load image using numpy

  Args:
      imagePath (str): image path 
      imageName (str): image name
      normalize (bool, optional): To normalize image or not. Defaults to True.

  Returns:
      numpy array: x,y,3 array
  """
  # print("{}{}{}".format(os.getcwd(), "\\"+imagePath, imageName))
  image_array = cv2.imread("{}{}{}".format(os.getcwd(), "\\"+imagePath, imageName))
  image_array = cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB)
  if normalize:
    image_array = image_array/255
    
  return image_array

# Extract the daata from 0 to amount from list and return it
def CropData(list, amount):
  if (len(list) < amount):
    amount = len(list)
  croppedList = list[:amount]
  del list[:amount]
  return croppedList

def LoadImages(imagePath, list):
  label = []
  data = []
  for entries in list:
    try:
      image = LoadImage(imagePath, entries[1])
      data.append(image) 
      label.append(entries[0]) 
    except: 
      print("Failed to load training image: ", entries[1])
  npLabel = np.array(label) 
  npData = np.array(data) 
  return npLabel, npData 

# Load a list of face mesh in landmark_pb2.NormalizedLandmarkList format
def LoadFaceMeshes(faceMesh, path, list, withFace = False):
  face = []
  label = []
  data = []
  for entry in list:
    # try:
    if withFace:
      face.append(LoadImage(path, entry[1], normalize = False)) # Load image
      data.append(faceMesh.process(face[-1])) # process it into face mesh and append into data
    else:
      data.append(faceMesh.process(LoadImage(path, entry[1], normalize = False))) # Load image, process it into face mesh, and append into data
    label.append(entry[0])
    # except:
      # print("Failed to load training image: ", entry[1])

  if withFace:
    return np.array(label), np.array(data), np.array(face)
  else:
    return np.array(label), np.array(data)

# Load face mesh data in numpy array of 
def LoadFaceMeshVertices(faceMesh, path, list):
  label = []
  data = []
  for entry in list:
    # load image, process it into normalizedlandmarklist format, obtain face data from the list, convert it into vertices
    image = LoadImage(path, entry[1], normalize = False)
    multiFaceLandmarks = faceMesh.process(image).multi_face_landmarks
    if (multiFaceLandmarks == None):
      print("{} multi-face landmarks is none".format(entry[1]))
      continue
    vertices = MultiLandmarkToVertices(multiFaceLandmarks[0])
    data.append(vertices)
    label.append(entry[0])
    # except:
      # print("Failed to load training image: ", entry[1])

def LoadVertices(path, list):
  label = []
  data = []
  for entry in list:
    vertices = np.load("{}{}{}.npy".format(os.getcwd(), "\\"+path, entry[1][:-4]))
    data.append(vertices)
    label.append(entry[0])
  
  return np.array(label), np.array(data)
  
def MultiLandmarkToVertices(multiLandmark):
  verticesList = []
  for idx, landmark in enumerate(multiLandmark.landmark):
    verticesList.append([landmark.x, landmark.y, landmark.z])

  return verticesList


def InitializeModel():
  pretrained_model = tf.keras.applications.MobileNetV3Large(input_shape=(224,224,3)) # Initializing model with mobile net V3 pretrained model

  # Initializing the input and output from the model, removing last layer
  base_input = pretrained_model.layers[0].input
  base_output = pretrained_model.layers[-2].output

  # Adding 3 more layers to output side
  final_output = layers.Dense(128)(base_output) # Adding new layers, to the output side
  final_output = layers.Activation('relu')(final_output) # activating layer
  final_output = layers.Dense(64)(final_output)
  final_output = layers.Activation('relu')(final_output) # activating layer
  final_output = layers.Dense(8, activation = 'softmax')(final_output) # 8 cuz there are 8 image classifications

  new_model = keras.Model(inputs = base_input, outputs = final_output)
  # new_model.summary()
  return new_model

def ConvertToGray(image):
  image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
  image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
  return image

def ScaleImage(image, width):
  ratio = image.shape[1]/width
  image = cv2.resize(image, (width, int(image.shape[0]/ratio)))
  return image

def DetectFace(image):
  face_roi = np.ndarray(1)
  faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
  grayImage = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
  faces = faceCascade.detectMultiScale(grayImage, 1.3, 5)
  for x,y,w,h in faces:
    roi_gray = grayImage[y:y+h, x:x+w]
    roi_color = image[y:y+h, x:x+w]
    cv2.rectangle(image, (x,y), (x+w, y+h), (255,0,0), 2)
    facess = faceCascade.detectMultiScale(roi_gray)
    if (len(facess) == 0):
      print("Face not detected")
    else:
      for (ex,ey,ew,eh) in facess:
        face_roi = roi_color[ey:ey+eh, ex:ex+ew]
  return face_roi

def ConvertToInput(image):
  input = ScaleImage(image, 224)
  input = np.expand_dims(input, axis = 0) ## to add fourth dimension to fit model input
  input = input/255
  return input

def GetResult(model, input):
  Predictions = model.predict(input)
  print(Predictions)
  result = np.argmax(Predictions)
  return emotionList[result]

def printDataSetLabels(dataSet):
    counterList = list(range(8))
    for label, name in dataSet:
        counterList[label] += 1
    print(counterList)
    
# Reset Keras Session
def reset_keras():
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    # try:
    #     del classifier # this is from global space - change this as you need
    # except:
    #     pass

    # use the same config as you used to create the session
    tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.9, visible_device_list="0")

class TimeHistory(keras.callbacks.Callback):
  def on_train_begin(self, logs={}):
    self.times = []

  def on_epoch_begin(self, batch, logs={}):
    self.epoch_time_start = time.time()

  def on_epoch_end(self, batch, logs={}):
    self.times.append(time.time() - self.epoch_time_start)

  def AverageTime(self):
    sum = 0
    for time in self.times:
      sum += time
    return sum/len(self.times)

def TestModel(model, testSetData):
  # test model
  lostSum = 0
  accuracySum = 0
  count = 0
  while(len(testSetData) != 0):
    # training data
    try:
      croppedList = CropData(testSetData, 100)
      label, data = LoadImages(TESTIMAGEPATH, croppedList)
      result = model.evaluate(data, label, batch_size = 1)
      lostSum += result[0]
      accuracySum += result[1]
    except:
      print("Failed to train data")

    count += 1
    reset_keras()
  
  return lostSum/count, accuracySum/count

def TestModelWithVertices(model, dataSet, tensorboard):
  # test model
  lostSum = 0
  accuracySum = 0
  count = 0
  while(len(dataSet) != 0):
    # training data
    try:
      croppedList = CropData(dataSet, 10000)
      label, data = LoadVertices(TESTLANDMARKPATH, croppedList)
      result = model.evaluate(data, label, batch_size = 1, callbacks =[tensorboard])
      lostSum += result[0]
      accuracySum += result[1]
    except:
      print("Failed to train data")

    count += 1
    reset_keras()
  
  return lostSum/count, accuracySum/count

def DrawFaceLandmark(label, image, landmarks):
  mp_drawing = mp.solutions.drawing_utils
  mp_drawing_styles = mp.solutions.drawing_styles
  mp_face_mesh = mp.solutions.face_mesh
  print(f'Face landmarks of {label}:')
  if not landmarks.multi_face_landmarks:
    print("unagle to locate face landmark for {}".format(label))
  annotated_image = image.copy()
  for face_landmarks in landmarks.multi_face_landmarks:
    mp_drawing.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks,
        connections=mp_face_mesh.FACEMESH_TESSELATION,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp_drawing_styles
        .get_default_face_mesh_tesselation_style())
    mp_drawing.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks,
        connections=mp_face_mesh.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp_drawing_styles
        .get_default_face_mesh_contours_style())
    mp_drawing.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks,
        connections=mp_face_mesh.FACEMESH_IRISES,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp_drawing_styles
        .get_default_face_mesh_iris_connections_style())
  plt.imshow(annotated_image)

## Setup directory and import image file names

In [3]:
# Get currect directory (os.getcwd() -> C:\Users\jazzt\src)

#-----------------------Start of code---------------------------
# initialise image names and label
mainTrainSet = LoadAllImageNames(TRAINLANDMARKPATH, catLimit=[5000]*8)
mainTestSet = LoadAllImageNames(TESTLANDMARKPATH, catLimit=[500]*8)

Files loaded:10000
Files loaded:20000
Files loaded:30000
Total images loaded:  37553
Images Loaded:  [5000, 5000, 5000, 5000, 5000, 3803, 5000, 3750]
Total images loaded:  3987
Images Loaded:  [499, 500, 498, 498, 498, 500, 499, 495]


In [ ]:
# reset learning rate callback to counter ReduceLROnPlateau
class ResetLR(tf.keras.callbacks.Callback):
    def __init__(self, learningRate):
        self.default_lr = learningRate
    def on_train_begin(self, logs={}):
        previous_lr = self.model.optimizer.lr.read_value()
        if previous_lr != self.default_lr:
            print("Resetting learning rate from {} to {}".format(previous_lr, self.default_lr))
            self.model.optimizer.lr.assign(self.default_lr)

# # making stopper to only start counting from 3 epochs onwards
# class CustomStopper(tf.keras.callbacks.EarlyStopping):
#     def __init__(self, monitor='val_loss',
#              min_delta=0, patience=3, verbose=0, mode='auto', start_epoch = 3): # add argument for starting epoch
#         super(CustomStopper, self).__init__()
#         self.start_epoch = start_epoch

#     def on_epoch_end(self, epoch, logs=None):
#         if epoch > self.start_epoch:
#             super().on_epoch_end(epoch, logs) 

In [5]:
SaveInformation = [] # Saving information as {name, bs, L1, L2, L3, L4, loss, accuracy}

batchSizes = [128, 256, 384]
layerSizes = [
  (512, 64, 64), (512, 192, 128), (512, 384, 256), 
  (764, 128, 48), (764, 128, 128), (764, 192, 128), (764, 192, 192), (764, 256, 256), 
  (1024, 256, 128), (1024, 384, 128)
]
epoch = 30
learningRate = 0.001

for batchSize in batchSizes:
  for layerSize in layerSizes:
    firstLayerSize = layerSize[0]
    secondLayerSize = layerSize[1]
    thirdLayerSize = layerSize[2]
    name = "flatten_{}epoch_{}lr_{}bs_{}_{}_{}_landmark".format(epoch, learningRate, batchSize, firstLayerSize, secondLayerSize, thirdLayerSize)
    FreezeLayerTrainAndTest(epoch, learningRate, name, batchSize, firstLayerSize, secondLayerSize, thirdLayerSize)


C:\Users\j.teoh\AppData\Local\miniconda3\envs\tflite-facial-expression\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


loading image
Epoch 1/30
198/198 [==============================] - 14s 65ms/step - loss: 2.0666 - accuracy: 0.1534 - val_loss: 2.0589 - val_accuracy: 0.1665
Epoch 2/30
198/198 [==============================] - 12s 59ms/step - loss: 2.0267 - accuracy: 0.1867 - val_loss: 2.0164 - val_accuracy: 0.1925
Epoch 3/30
198/198 [==============================] - 12s 63ms/step - loss: 1.9883 - accuracy: 0.2145 - val_loss: 2.0035 - val_accuracy: 0.2000
Epoch 4/30
198/198 [==============================] - 13s 64ms/step - loss: 1.9773 - accuracy: 0.2201 - val_loss: 1.9852 - val_accuracy: 0.2145
Epoch 5/30
198/198 [==============================] - 13s 66ms/step - loss: 1.9602 - accuracy: 0.2304 - val_loss: 1.9727 - val_accuracy: 0.2242
Epoch 6/30
198/198 [==============================] - 13s 66ms/step - loss: 1.9582 - accuracy: 0.2314 - val_loss: 1.9358 - val_accuracy: 0.2546
Epoch 7/30
198/198 [==============================] - 13s 66ms/step - loss: 1.9449 - accuracy: 0.2425 - val_loss: 1.9253 -

KeyboardInterrupt: 

In [ ]:
batchSizes = [128, 256, 384]
firstLayerSizes = [512, 764, 1024, 1536, 2048]
secondLayerSizes = [64, 128, 192, 256, 384, 512, 764, 1024]
thirdLayerSizes = [64, 128, 192, 256, 384, 512, 764, 1024]
fourthLayerSizes = [16, 32, 48, 64, 128, 192, 256]
epoch = 30
learningRate = 0.001

for batchSize in batchSizes:
  for firstLayerSize in firstLayerSizes:
    for secondLayerSize in secondLayerSizes:
      if secondLayerSize > firstLayerSize:
        continue
      for thirdLayerSize in thirdLayerSizes:
        if (thirdLayerSize > secondLayerSize):
          continue
        for fourthLayerSize in fourthLayerSizes:
          if (fourthLayerSize > thirdLayerSize):
            continue
          name = "freeze_{}epoch_{}lr_{}bs_{}_{}_{}_{}_landmark".format(epoch, learningRate, batchSize, firstLayerSize, secondLayerSize, thirdLayerSize, fourthLayerSize)
          FreezeLayerTrainAndTest(epoch, learningRate, name, batchSize, firstLayerSize, secondLayerSize, thirdLayerSize, fourthLayerSize)


In [4]:

def FreezeLayerTrainAndTest(epoch, learningRate, modelName, batchSize, firstLayerSize, secondLayerSize, thirdLayerSize):
  """Train and test a model. This function's input should be customized to accept different parameters, function body should be customized to these parameters to train models of variety of paramters.

  Args:
      modelName (str): name of model
  """
  #-------------------------------------------------
  # Building model
  #-------------------------------------------------
  # Adding 3 more layers to output side
  inputs = tf.keras.Input(shape=(478,3))
  base = tf.keras.layers.Dense(firstLayerSize, activation='relu')(inputs)
  base = tf.keras.layers.Dense(secondLayerSize, activation='relu')(base)
  base = tf.keras.layers.Dense(thirdLayerSize, activation='relu')(base)
  base = tf.keras.layers.Flatten()(base)
  outputs = tf.keras.layers.Dense(8, activation='softmax')(base)
  
  model = tf.keras.Model(inputs=inputs, outputs=outputs)
  model.compile(loss = "sparse_categorical_crossentropy", optimizer = keras.optimizers.Adam(lr = learningRate), metrics=["accuracy"])

  # ------------------ Callbacks ------------------
  earlyStop = EarlyStopping(monitor='val_loss', patience = 4, restore_best_weights = True)
  # earlyStopAcc = EarlyStopping(monitor='val_accuracy', patience = 1, restore_best_weights = True)
  # reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, patience = 2, min_lr = 0.0002)
  # reset_lr = ResetLR(learningRate)

  # ------------------ Training Model ------------------
  time_callback = TimeHistory()
  trainDataSet = mainTrainSet.copy()

  # Initializing model fit params
  # batchSize = 4
  imgPerIter = batchSize*256

  count = 0
  iter = 0
  while(len(trainDataSet) != 0):
    iter += 1
    tensorboard = TensorBoard(log_dir = "logs/{}_it{}".format(modelName, iter))
    croppedList = []
    try:
      print("loading image") # Crop and load images in
      croppedList = CropData(trainDataSet, imgPerIter)
      label, data = LoadVertices(TRAINLANDMARKPATH, croppedList)
      # Training model
      model.fit(data, label, epochs = epoch, validation_split = 0.2, batch_size = batchSize, callbacks = [time_callback, tensorboard, earlyStop]) # earlyStop, earlyStopAcc, reduce_lr, reset_lr
      print(time_callback.AverageTime())
    except RuntimeError as e:
      print(e)
    
    # Saving weights
    model.save_weights('{}.h5'.format(modelName))
    
    # Print summary of current iteration
    count = count + len(croppedList)
    timeLeft = len(trainDataSet)/imgPerIter * time_callback.AverageTime()
    print("trained image count: ", count)
    print("Images Left: ", len(trainDataSet))
    print("Estimated completion time: ", str(timedelta(seconds=timeLeft)))
    
    reset_keras()
  print("---------------- {} Training Completed ------------------".format(modelName))

  # ---------------- Testing Model -----------------
  # initialise image names and label
  testSetData = mainTestSet.copy()

  # test model
  loss, accuracy = TestModelWithVertices(model, testSetData, tensorboard)

  # Saving test information on notebook
  SaveInformation.append({"name": modelName, 
                          "batch": batchSize, 
                          "L1": firstLayerSize, 
                          "L2": secondLayerSize, 
                          "L3": thirdLayerSize, 
                          "loss": loss, 
                          "accuracy": accuracy })

  # Saving test informations
  modelSummary = '''
  model name: {}
  average lost: {}
  average accuracy: {}
            '''.format(modelName, loss, accuracy)
      
  print("==========FINISH TESTING===========\n{}".format(modelSummary))
  
  file = open("training_summaries.txt", "a")
  file.write(modelSummary + "\n\n")

In [ ]:
model.summary()

In [ ]:
for info in SaveInformation:
    print(info['accuracy'])

In [10]:
def FreezeLayerTrainAndTesttesting(epoch, learningRate, modelName, batchSize, firstLayerSize, secondLayerSize, thirdLayerSize, fourthLayerSize):
  """Train and test a model. This function's input should be customized to accept different parameters, function body should be customized to these parameters to train models of variety of paramters.

  Args:
      modelName (str): name of model
  """
  #-------------------------------------------------
  # Building model
  #-------------------------------------------------
  # Adding 3 more layers to output side
  inputs = tf.keras.Input(shape=(478,3))
  base = tf.keras.layers.Dense(firstLayerSize, activation='relu')(inputs)
  base = tf.keras.layers.Dense(secondLayerSize, activation='relu')(base)
  base = tf.keras.layers.Flatten()(base)
  base = tf.keras.layers.Dense(thirdLayerSize, activation='relu')(base)
  base = tf.keras.layers.Dense(fourthLayerSize, activation='relu')(base)
  outputs = tf.keras.layers.Dense(8, activation='softmax')(base)
  
  model = tf.keras.Model(inputs=inputs, outputs=outputs)
  model.compile(loss = "sparse_categorical_crossentropy", optimizer = keras.optimizers.Adam(lr = learningRate), metrics=["accuracy"])

  # ------------------ Callbacks ------------------
  # earlyStop = EarlyStopping(monitor='val_loss', patience = 4, restore_best_weights = True)
  # earlyStopAcc = EarlyStopping(monitor='val_accuracy', patience = 1, restore_best_weights = True)
  # reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, patience = 2, min_lr = 0.0002)
  # reset_lr = ResetLR(learningRate)

  # ------------------ Training Model ------------------
  time_callback = TimeHistory()
  trainDataSet = mainTrainSet.copy()

  # Initializing model fit params
  # batchSize = 4
  imgPerIter = batchSize*500

  count = 0
  iter = 0
  while(len(trainDataSet) != 0):
    iter += 1
    tensorboard = TensorBoard(log_dir = "logs/{}_it{}".format(modelName, iter))
    croppedList = []
    try:
      print("loading image") # Crop and load images in
      croppedList = CropData(trainDataSet, imgPerIter)
      label, data = LoadVertices(TRAINLANDMARKPATH, croppedList)
      # Training model
      model.fit(data, label, epochs = epoch, validation_split = 0.2, batch_size = batchSize, callbacks = [time_callback, tensorboard]) # earlyStop, earlyStopAcc, reduce_lr, reset_lr
      print(time_callback.AverageTime())
    except RuntimeError as e:
      print(e)
    
    # Saving weights
    model.save_weights('{}.h5'.format(modelName))
    
    # Print summary of current iteration
    count = count + len(croppedList)
    timeLeft = len(trainDataSet)/imgPerIter * time_callback.AverageTime()
    print("trained image count: ", count)
    print("Images Left: ", len(trainDataSet))
    print("Estimated completion time: ", str(timedelta(seconds=timeLeft)))
    
    reset_keras()
  print("---------------- {} Training Completed ------------------".format(modelName))

  # ---------------- Testing Model -----------------
  # initialise image names and label
  testSetData = mainTestSet.copy()

  # test model
  loss, accuracy = TestModelWithVertices(model, testSetData, tensorboard)

  # Saving test information on notebook
  SaveInformation.append({"name": modelName, 
                          "batch": batchSize, 
                          "L1": firstLayerSize, 
                          "L2": secondLayerSize, 
                          "L3": thirdLayerSize, 
                          "L3": fourthLayerSize, 
                          "loss": loss, 
                          "accuracy": accuracy })

  # Saving test informations
  modelSummary = '''
  model name: {}
  average lost: {}
  average accuracy: {}
            '''.format(modelName, loss, accuracy)
      
  print("==========FINISH TESTING===========\n{}".format(modelSummary))
  
  file = open("training_summaries.txt", "a")
  file.write(modelSummary + "\n\n")
    

In [ ]:
SaveInformation = []
batchSizes = [128, 96]
firstLayerSizes = [196]
secondLayerSizes = [64]
thirdLayerSizes = [512]
fourthLayerSizes = [64]
epoch = 40
learningRate = 0.002
for batchSize in batchSizes:
  for firstLayerSize in firstLayerSizes:
    for secondLayerSize in secondLayerSizes:
      if secondLayerSize > firstLayerSize:
        continue
      for thirdLayerSize in thirdLayerSizes:
        for fourthLayerSize in fourthLayerSizes:
          if (fourthLayerSize > thirdLayerSize):
            continue
          name = "ffdffFinal_{}epoch_{}lr_{}bs_{}_{}_{}_{}_landmark".format(epoch, learningRate, batchSize, firstLayerSize, secondLayerSize, thirdLayerSize, fourthLayerSize)
          FreezeLayerTrainAndTesttesting(epoch, learningRate, name, batchSize, firstLayerSize, secondLayerSize, thirdLayerSize, fourthLayerSize)

C:\Users\jazzt\anaconda3\envs\tflite-facial-expression\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


loading image
Epoch 1/40
235/235 [==============================] - 18s 75ms/step - loss: 2.0326 - accuracy: 0.1914 - val_loss: 1.9868 - val_accuracy: 0.2198
Epoch 2/40
235/235 [==============================] - 18s 76ms/step - loss: 1.9478 - accuracy: 0.2386 - val_loss: 1.9404 - val_accuracy: 0.2396
Epoch 3/40
235/235 [==============================] - 19s 81ms/step - loss: 1.9361 - accuracy: 0.2446 - val_loss: 1.9304 - val_accuracy: 0.2459
Epoch 4/40
235/235 [==============================] - 19s 81ms/step - loss: 1.9255 - accuracy: 0.2521 - val_loss: 1.9120 - val_accuracy: 0.2617
Epoch 5/40
235/235 [==============================] - 19s 80ms/step - loss: 1.9185 - accuracy: 0.2575 - val_loss: 1.9109 - val_accuracy: 0.2624
Epoch 6/40
235/235 [==============================] - 19s 81ms/step - loss: 1.9034 - accuracy: 0.2714 - val_loss: 1.9051 - val_accuracy: 0.2641
Epoch 7/40
235/235 [==============================] - 19s 79ms/step - loss: 1.8837 - accuracy: 0.2813 - val_loss: 1.8934 -